In [ ]:
## This file implements neural networks for p11presabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p11presabsSTCC_qual.csv')
df.shape

(253, 955)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
      ..
248    2
249    1
250    2
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_10026,group_10028,group_10211,group_2860,group_5746,group_7042,group_867,ST,CC,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [6]:
df['pheno'].value_counts()

2    181
1     47
0     25
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 954)

In [9]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_10026,group_10028,group_10211,group_2860,group_5746,group_7042,group_867,ST,CC,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 954) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 181), (1, 181), (2, 181)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,SR1129,2
1,NRS185,2
2,NRS243,1
3,BCH-SA-04,0
4,504,1
...,...,...
158,CFBREBSa131,2
159,CFBREBSa133,1
160,NRS256,2
161,GA48963,1


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 430us/step - loss: 1.4423 - accuracy: 0.3763 - val_loss: 1.0344 - val_accuracy: 0.4724
Epoch 2/100
380/380 [==============================] - 0s 133us/step - loss: 0.9962 - accuracy: 0.5368 - val_loss: 0.9052 - val_accuracy: 0.6196
Epoch 3/100
380/380 [==============================] - 0s 152us/step - loss: 0.9405 - accuracy: 0.6132 - val_loss: 0.9476 - val_accuracy: 0.6626
Epoch 4/100
380/380 [==============================] - 0s 167us/step - loss: 0.8408 - accuracy: 0.6158 - val_loss: 0.8174 - val_accuracy: 0.6319
Epoch 5/100
380/380 [==============================] - 0s 171us/step - loss: 0.7786 - accuracy: 0.6553 - val_loss: 0.7769 - val_accuracy: 0.6442
Epoch 6/100
380/380 [==============================] - 0s 226us/step - loss: 0.7238 - accuracy: 0.6447 - val_loss: 0.7710 - val_accuracy: 0.6810
Epoch 7/100
380/380 [==============================] - 0s 226us/step - loss: 0.6947 

Epoch 57/100
380/380 [==============================] - 0s 155us/step - loss: 0.1904 - accuracy: 0.9368 - val_loss: 0.3735 - val_accuracy: 0.8957
Epoch 58/100
380/380 [==============================] - 0s 120us/step - loss: 0.1723 - accuracy: 0.9500 - val_loss: 0.4602 - val_accuracy: 0.8834
Epoch 59/100
380/380 [==============================] - 0s 175us/step - loss: 0.2091 - accuracy: 0.9447 - val_loss: 0.4077 - val_accuracy: 0.8650
Epoch 60/100
380/380 [==============================] - 0s 295us/step - loss: 0.1935 - accuracy: 0.9474 - val_loss: 0.4881 - val_accuracy: 0.8528
Epoch 61/100
380/380 [==============================] - 0s 277us/step - loss: 0.2296 - accuracy: 0.9447 - val_loss: 0.4431 - val_accuracy: 0.8650
Epoch 62/100
380/380 [==============================] - 0s 284us/step - loss: 0.1729 - accuracy: 0.9474 - val_loss: 0.3723 - val_accuracy: 0.8957
Epoch 63/100
380/380 [==============================] - 0s 209us/step - loss: 0.1759 - accuracy: 0.9474 - val_loss: 0.3964 -

In [33]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

163/163 [==============================] - 0s 114us/step
over-sampling test accuracy: 89.57%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 1, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 1, 1, 0,
       0, 2, 0, 0, 1, 1, 0, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 2,
       1, 1, 2, 1, 0, 0, 0, 2, 1, 1, 1, 2, 2, 2, 0, 2, 0, 1, 0, 1, 1, 2,
       0, 0, 1, 2, 0, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 0, 0, 1,
       1, 2, 0, 1, 2, 0, 1, 2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       2, 1, 1, 1, 2, 0, 0, 1, 1, 1, 0, 2, 1, 2, 0, 2, 2, 0, 2, 0, 0, 1,
       2, 1, 2, 2, 0, 0, 2, 0, 1, 2, 0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0,
       1, 2, 0, 0, 2, 1, 2, 1, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,SR1129,2,0
1,NRS185,2,2
2,NRS243,1,1
3,BCH-SA-04,0,0
4,504,1,1
...,...,...,...
158,CFBREBSa131,2,2
159,CFBREBSa133,1,1
160,NRS256,2,2
161,GA48963,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,7.458099e-01,4.042330e-03,0.250148
1,3.313246e-04,2.231654e-03,0.997437
2,8.059137e-05,9.862996e-01,0.013620
3,9.979766e-01,1.724426e-06,0.002022
4,3.604608e-04,9.862773e-01,0.013362
...,...,...,...
158,5.107138e-07,3.921908e-07,0.999999
159,4.293338e-04,9.504268e-01,0.049144
160,6.359447e-04,2.067653e-03,0.997296
161,3.365450e-05,9.926442e-01,0.007322


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p11pST.csv", index = False,
         header=None)

In [37]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 188us/step - loss: 0.0915 - accuracy: 0.9684 - val_loss: 0.2984 - val_accuracy: 0.9202
Epoch 2/100
380/380 [==============================] - 0s 152us/step - loss: 0.0916 - accuracy: 0.9632 - val_loss: 0.3110 - val_accuracy: 0.9080
Epoch 3/100
380/380 [==============================] - 0s 153us/step - loss: 0.1246 - accuracy: 0.9553 - val_loss: 0.3622 - val_accuracy: 0.8896
Epoch 4/100
380/380 [==============================] - 0s 169us/step - loss: 0.1503 - accuracy: 0.9289 - val_loss: 0.4260 - val_accuracy: 0.8896
Epoch 5/100
380/380 [==============================] - 0s 129us/step - loss: 0.1227 - accuracy: 0.9553 - val_loss: 0.4931 - val_accuracy: 0.8589
Epoch 6/100
380/380 [==============================] - 0s 136us/step - loss: 0.1411 - accuracy: 0.9474 - val_loss: 0.4209 - val_accuracy: 0.8466
Epoch 7/100
380/380 [==============================] - 0s 123us/step - loss: 0.1387 

Epoch 57/100
380/380 [==============================] - 0s 233us/step - loss: 0.0538 - accuracy: 0.9763 - val_loss: 0.4437 - val_accuracy: 0.8712
Epoch 58/100
380/380 [==============================] - 0s 174us/step - loss: 0.0588 - accuracy: 0.9711 - val_loss: 0.3741 - val_accuracy: 0.9018
Epoch 59/100
380/380 [==============================] - 0s 164us/step - loss: 0.0512 - accuracy: 0.9816 - val_loss: 0.3441 - val_accuracy: 0.9080
Epoch 60/100
380/380 [==============================] - 0s 246us/step - loss: 0.0492 - accuracy: 0.9816 - val_loss: 0.3876 - val_accuracy: 0.8957
Epoch 61/100
380/380 [==============================] - 0s 211us/step - loss: 0.0630 - accuracy: 0.9711 - val_loss: 0.3175 - val_accuracy: 0.9080
Epoch 62/100
380/380 [==============================] - 0s 223us/step - loss: 0.0608 - accuracy: 0.9789 - val_loss: 0.3260 - val_accuracy: 0.9080
Epoch 63/100
380/380 [==============================] - 0s 246us/step - loss: 0.0638 - accuracy: 0.9658 - val_loss: 0.3189 -

In [38]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 96.95%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p11presabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.45809850e-01, 4.04232970e-03, 2.50147850e-01],
       [3.31324580e-04, 2.23165380e-03, 9.97437000e-01],
       [8.05913660e-05, 9.86299600e-01, 1.36198530e-02],
       [9.97976600e-01, 1.72442580e-06, 2.02174600e-03],
       [3.60460840e-04, 9.86277300e-01, 1.33621900e-02],
       [9.27506700e-01, 2.92116140e-02, 4.32817230e-02],
       [9.97403200e-01, 5.61293740e-04, 2.03540480e-03],
       [9.31508960e-01, 1.55653310e-04, 6.83353500e-02],
       [1.75666000e-02, 6.27354700e-01, 3.55078700e-01],
       [3.75761370e-05, 9.98351450e-01, 1.61093470e-03],
       [9.38545100e-06, 1.65671540e-03, 9.98333900e-01],
       [9.93878800e-01, 4.75678640e-03, 1.36438130e-03],
       [7.00319230e-01, 1.62725100e-01, 1.36955660e-01],
       [1.96539000e-02, 2.34565590e-02, 9.56889570e-01],
       [9.93878800e-01, 4.75678640e-03, 1.36438130e-03],
       [9.97550800e-01, 1.54193510e-06, 2.44762300e-03],
       [8.12448900e-05, 7.73149640e-03, 9.92187260e-01],
       [8.06043100e-04, 7.18011

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.976633305532388

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.976633305532388

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,NRS027,0
1,CFBRSa07,0
2,CFBRSa27,1
3,504,1
4,CFBREBSa129,0
...,...,...
158,SR3569,2
159,NRS243,1
160,GA48963,1
161,504,1


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 410us/step - loss: 3.9275 - accuracy: 0.3605 - val_loss: 5.6058 - val_accuracy: 0.3988
Epoch 2/100
380/380 [==============================] - 0s 104us/step - loss: 2.6435 - accuracy: 0.3132 - val_loss: 4.2434 - val_accuracy: 0.3558
Epoch 3/100
380/380 [==============================] - 0s 119us/step - loss: 1.8634 - accuracy: 0.3895 - val_loss: 3.0833 - val_accuracy: 0.3988
Epoch 4/100
380/380 [==============================] - 0s 127us/step - loss: 1.4835 - accuracy: 0.4526 - val_loss: 2.4358 - val_accuracy: 0.4540
Epoch 5/100
380/380 [==============================] - 0s 166us/step - loss: 1.1900 - accuracy: 0.5289 - val_loss: 1.9104 - val_accuracy: 0.4724
Epoch 6/100
380/380 [==============================] - 0s 160us/step - loss: 1.1021 - accuracy: 0.5789 - val_loss: 1.5635 - val_accuracy: 0.4785
Epoch 7/100
380/380 [==============================] - 0s 149us/step - loss: 0.9683 

Epoch 57/100
380/380 [==============================] - 0s 195us/step - loss: 0.2377 - accuracy: 0.9184 - val_loss: 0.3990 - val_accuracy: 0.8834
Epoch 58/100
380/380 [==============================] - 0s 230us/step - loss: 0.2293 - accuracy: 0.9184 - val_loss: 0.4213 - val_accuracy: 0.8896
Epoch 59/100
380/380 [==============================] - 0s 248us/step - loss: 0.2253 - accuracy: 0.9289 - val_loss: 0.3639 - val_accuracy: 0.8650
Epoch 60/100
380/380 [==============================] - 0s 215us/step - loss: 0.2219 - accuracy: 0.9263 - val_loss: 0.3938 - val_accuracy: 0.8957
Epoch 61/100
380/380 [==============================] - 0s 101us/step - loss: 0.2220 - accuracy: 0.9211 - val_loss: 0.3639 - val_accuracy: 0.8834
Epoch 62/100
380/380 [==============================] - 0s 87us/step - loss: 0.2130 - accuracy: 0.9237 - val_loss: 0.3691 - val_accuracy: 0.9080
Epoch 63/100
380/380 [==============================] - 0s 84us/step - loss: 0.2247 - accuracy: 0.9079 - val_loss: 0.3981 - v

In [62]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

163/163 [==============================] - 0s 117us/step
over-sampling test accuracy: 91.41%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 0,
       1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 0, 2, 1,
       0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 2, 1, 2, 1, 0, 1, 1, 1, 0,
       2, 2, 2, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 2, 1, 2, 1, 0,
       1, 1, 0, 0, 0, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 0, 0, 2, 0, 0,
       2, 0, 2, 2, 1, 1, 1, 2, 1, 2, 0, 2, 0, 1, 0, 2, 0, 2, 1, 0, 0, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS027,0,0
1,CFBRSa07,0,0
2,CFBRSa27,1,1
3,504,1,1
4,CFBREBSa129,0,0
...,...,...,...
158,SR3569,2,1
159,NRS243,1,1
160,GA48963,1,1
161,504,1,1


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.998794,0.000903,0.000303
1,0.996671,0.003000,0.000329
2,0.034051,0.827807,0.138142
3,0.007163,0.935975,0.056862
4,0.999048,0.000870,0.000081
...,...,...,...
158,0.009522,0.608981,0.381497
159,0.000305,0.957949,0.041746
160,0.000363,0.941257,0.058380
161,0.007163,0.935975,0.056862


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p11pST.csv", index = False,
         header=None)

In [66]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 204us/step - loss: 0.1113 - accuracy: 0.9632 - val_loss: 0.4424 - val_accuracy: 0.8957
Epoch 2/100
380/380 [==============================] - 0s 125us/step - loss: 0.1064 - accuracy: 0.9632 - val_loss: 0.3629 - val_accuracy: 0.8957
Epoch 3/100
380/380 [==============================] - 0s 150us/step - loss: 0.1076 - accuracy: 0.9632 - val_loss: 0.3570 - val_accuracy: 0.9080
Epoch 4/100
380/380 [==============================] - 0s 138us/step - loss: 0.1048 - accuracy: 0.9632 - val_loss: 0.4801 - val_accuracy: 0.8834
Epoch 5/100
380/380 [==============================] - 0s 127us/step - loss: 0.1136 - accuracy: 0.9579 - val_loss: 0.3654 - val_accuracy: 0.9080
Epoch 6/100
380/380 [==============================] - 0s 141us/step - loss: 0.1054 - accuracy: 0.9658 - val_loss: 0.3911 - val_accuracy: 0.8957
Epoch 7/100
380/380 [==============================] - 0s 129us/step - loss: 0.0993 

Epoch 57/100
380/380 [==============================] - 0s 99us/step - loss: 0.0805 - accuracy: 0.9684 - val_loss: 0.6558 - val_accuracy: 0.8712
Epoch 58/100
380/380 [==============================] - 0s 95us/step - loss: 0.0838 - accuracy: 0.9684 - val_loss: 0.6620 - val_accuracy: 0.8896
Epoch 59/100
380/380 [==============================] - 0s 99us/step - loss: 0.0800 - accuracy: 0.9737 - val_loss: 0.5494 - val_accuracy: 0.9141
Epoch 60/100
380/380 [==============================] - 0s 102us/step - loss: 0.0769 - accuracy: 0.9711 - val_loss: 0.6479 - val_accuracy: 0.8834
Epoch 61/100
380/380 [==============================] - 0s 102us/step - loss: 0.0731 - accuracy: 0.9789 - val_loss: 0.5620 - val_accuracy: 0.9141
Epoch 62/100
380/380 [==============================] - 0s 359us/step - loss: 0.0685 - accuracy: 0.9737 - val_loss: 0.6522 - val_accuracy: 0.8957
Epoch 63/100
380/380 [==============================] - 0s 139us/step - loss: 0.0657 - accuracy: 0.9763 - val_loss: 0.5741 - va

In [67]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 96.85%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p11presabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.9879396e-01, 9.0337130e-04, 3.0261776e-04],
       [9.9667080e-01, 2.9997230e-03, 3.2942856e-04],
       [3.4051300e-02, 8.2780695e-01, 1.3814175e-01],
       [7.1629840e-03, 9.3597496e-01, 5.6861997e-02],
       [9.9904805e-01, 8.7045950e-04, 8.1462895e-05],
       [9.3585515e-01, 4.0976547e-02, 2.3168314e-02],
       [9.9904805e-01, 8.7045950e-04, 8.1462895e-05],
       [3.5724414e-03, 9.2506860e-01, 7.1358990e-02],
       [3.9170510e-02, 8.9149505e-01, 6.9334470e-02],
       [3.3237330e-02, 9.3049830e-01, 3.6264442e-02],
       [7.2584805e-07, 2.0911873e-03, 9.9790810e-01],
       [9.9993060e-01, 5.1309958e-06, 6.4307810e-05],
       [1.2467727e-02, 3.1199074e-01, 6.7554146e-01],
       [7.4236420e-03, 9.5181200e-01, 4.0764310e-02],
       [5.1882220e-04, 9.9699030e-01, 2.4908240e-03],
       [2.1552147e-08, 1.1833746e-05, 9.9998820e-01],
       [9.1492676e-04, 9.6463670e-01, 3.4448408e-02],
       [1.3996516e-03, 9.6731836e-01, 3.1282004e-02],
       [1.6539750e-02, 7.057

In [33]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.973249312698854

In [34]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.973249312698854

In [35]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [36]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [37]:
dat3

,0,test
0,NRS149,0
1,EUH13,0
2,NRS106,2
3,NRS214,1
4,CFBREBSa129,0
...,...,...
158,NRS027,0
159,CFBRSa70,2
160,CFBREBSa130,0
161,NRS214,1


In [38]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [39]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 549us/step - loss: 1.7203 - accuracy: 0.3711 - val_loss: 0.9571 - val_accuracy: 0.5951
Epoch 2/100
380/380 [==============================] - 0s 185us/step - loss: 1.0711 - accuracy: 0.5895 - val_loss: 1.1015 - val_accuracy: 0.6319
Epoch 3/100
380/380 [==============================] - 0s 197us/step - loss: 0.9029 - accuracy: 0.6342 - val_loss: 0.7664 - val_accuracy: 0.6626
Epoch 4/100
380/380 [==============================] - 0s 168us/step - loss: 0.7758 - accuracy: 0.6500 - val_loss: 0.8073 - val_accuracy: 0.6810
Epoch 5/100
380/380 [==============================] - 0s 133us/step - loss: 0.7581 - accuracy: 0.6763 - val_loss: 0.7102 - val_accuracy: 0.6871
Epoch 6/100
380/380 [==============================] - 0s 346us/step - loss: 0.7388 - accuracy: 0.7132 - val_loss: 1.0132 - val_accuracy: 0.6748
Epoch 7/100
380/380 [==============================] - 0s 135us/step - loss: 0.7215 

Epoch 57/100
380/380 [==============================] - 0s 117us/step - loss: 0.1625 - accuracy: 0.9474 - val_loss: 0.4954 - val_accuracy: 0.9018
Epoch 58/100
380/380 [==============================] - 0s 117us/step - loss: 0.1990 - accuracy: 0.9500 - val_loss: 0.4463 - val_accuracy: 0.8712
Epoch 59/100
380/380 [==============================] - 0s 160us/step - loss: 0.1758 - accuracy: 0.9474 - val_loss: 0.6068 - val_accuracy: 0.8896
Epoch 60/100
380/380 [==============================] - 0s 127us/step - loss: 0.1490 - accuracy: 0.9658 - val_loss: 0.4388 - val_accuracy: 0.8773
Epoch 61/100
380/380 [==============================] - 0s 112us/step - loss: 0.1189 - accuracy: 0.9684 - val_loss: 0.3431 - val_accuracy: 0.9018
Epoch 62/100
380/380 [==============================] - 0s 110us/step - loss: 0.1351 - accuracy: 0.9421 - val_loss: 0.4391 - val_accuracy: 0.8712
Epoch 63/100
380/380 [==============================] - 0s 111us/step - loss: 0.1506 - accuracy: 0.9579 - val_loss: 0.4676 -

In [88]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

163/163 [==============================] - 0s 133us/step
over-sampling test accuracy: 90.80%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 1, 2, 1, 2, 0, 1, 1, 0, 0, 2, 1, 2,
       0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 1, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 1,
       1, 0, 2, 1, 0, 0, 2, 0, 0, 0, 1, 1, 2, 1, 2, 1, 2, 1, 0, 1, 1, 0,
       0, 2, 0, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 1, 1, 2, 2, 2, 0, 0, 1,
       2, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 2, 1, 1, 1,
       0, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 0,
       1, 1, 0, 2, 0, 0, 0, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS149,0,0
1,EUH13,0,0
2,NRS106,2,0
3,NRS214,1,1
4,CFBREBSa129,0,0
...,...,...,...
158,NRS027,0,0
159,CFBRSa70,2,1
160,CFBREBSa130,0,0
161,NRS214,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.995607,0.004015,3.779488e-04
1,0.997946,0.002054,2.988466e-09
2,0.988839,0.009925,1.236040e-03
3,0.002792,0.993562,3.645902e-03
4,0.999961,0.000039,4.804003e-07
...,...,...,...
158,0.999872,0.000099,2.939901e-05
159,0.254730,0.739795,5.474732e-03
160,0.999541,0.000243,2.159090e-04
161,0.002792,0.993562,3.645902e-03


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p11pST.csv", index = False,
         header=None)

In [92]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 185us/step - loss: 0.0804 - accuracy: 0.9684 - val_loss: 0.3838 - val_accuracy: 0.8957
Epoch 2/100
380/380 [==============================] - 0s 146us/step - loss: 0.1294 - accuracy: 0.9526 - val_loss: 0.5415 - val_accuracy: 0.8650
Epoch 3/100
380/380 [==============================] - 0s 143us/step - loss: 0.1452 - accuracy: 0.9447 - val_loss: 0.4494 - val_accuracy: 0.8650
Epoch 4/100
380/380 [==============================] - 0s 119us/step - loss: 0.1224 - accuracy: 0.9658 - val_loss: 0.5133 - val_accuracy: 0.8712
Epoch 5/100
380/380 [==============================] - 0s 133us/step - loss: 0.0841 - accuracy: 0.9789 - val_loss: 0.6128 - val_accuracy: 0.8221
Epoch 6/100
380/380 [==============================] - 0s 122us/step - loss: 0.0704 - accuracy: 0.9763 - val_loss: 0.5470 - val_accuracy: 0.8712
Epoch 7/100
380/380 [==============================] - 0s 132us/step - loss: 0.0643 

Epoch 57/100
380/380 [==============================] - 0s 278us/step - loss: 0.0404 - accuracy: 0.9895 - val_loss: 0.4190 - val_accuracy: 0.8896
Epoch 58/100
380/380 [==============================] - 0s 288us/step - loss: 0.0413 - accuracy: 0.9868 - val_loss: 0.5719 - val_accuracy: 0.8773
Epoch 59/100
380/380 [==============================] - 0s 202us/step - loss: 0.0403 - accuracy: 0.9895 - val_loss: 0.6281 - val_accuracy: 0.8712
Epoch 60/100
380/380 [==============================] - 0s 199us/step - loss: 0.0734 - accuracy: 0.9711 - val_loss: 0.4930 - val_accuracy: 0.8834
Epoch 61/100
380/380 [==============================] - 0s 156us/step - loss: 0.0505 - accuracy: 0.9842 - val_loss: 0.4648 - val_accuracy: 0.8957
Epoch 62/100
380/380 [==============================] - 0s 230us/step - loss: 0.0511 - accuracy: 0.9842 - val_loss: 0.4731 - val_accuracy: 0.8957
Epoch 63/100
380/380 [==============================] - 0s 261us/step - loss: 0.0552 - accuracy: 0.9868 - val_loss: 0.5033 -

In [93]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 97.89%


In [40]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [41]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [42]:
y_prob3 = df_proba3[df_proba3['phage']=='p11presabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.95606960e-01, 4.01509600e-03, 3.77948830e-04],
       [9.97945960e-01, 2.05399350e-03, 2.98846570e-09],
       [9.88838800e-01, 9.92516800e-03, 1.23604010e-03],
       [2.79162400e-03, 9.93562460e-01, 3.64590180e-03],
       [9.99960800e-01, 3.87630130e-05, 4.80400330e-07],
       [1.01636810e-04, 1.35213510e-01, 8.64684900e-01],
       [9.99871850e-01, 9.87020100e-05, 2.93990070e-05],
       [9.37966640e-01, 5.81597600e-02, 3.87363860e-03],
       [4.70202200e-03, 6.74733500e-02, 9.27824600e-01],
       [9.93875440e-01, 6.11242370e-03, 1.20862480e-05],
       [1.65065940e-02, 9.78892200e-01, 4.60123220e-03],
       [1.06632440e-08, 3.11665170e-05, 9.99968770e-01],
       [3.08829390e-06, 9.98231700e-01, 1.76521110e-03],
       [1.16661930e-11, 7.00288800e-09, 1.00000000e+00],
       [9.99004540e-01, 9.93439900e-04, 1.97578130e-06],
       [1.80316440e-03, 9.89571800e-01, 8.62508100e-03],
       [8.32595500e-03, 9.26772400e-01, 6.49015750e-02],
       [9.37966640e-01, 5.81597

In [43]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9630983638628899

In [44]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9630983638628899

In [45]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [46]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [47]:
dat4

,0,test
0,SR2852,2
1,CFBREBSa138,0
2,BCH-SA-12,0
3,EUH13,0
4,EUH13,0
...,...,...
158,NRS036,1
159,CA105,1
160,CFBRSa51,1
161,NRS102,1


In [48]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [49]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 533us/step - loss: 2.4192 - accuracy: 0.3684 - val_loss: 1.3242 - val_accuracy: 0.5092
Epoch 2/100
380/380 [==============================] - 0s 172us/step - loss: 1.1238 - accuracy: 0.4816 - val_loss: 1.0534 - val_accuracy: 0.4847
Epoch 3/100
380/380 [==============================] - 0s 198us/step - loss: 0.9654 - accuracy: 0.5553 - val_loss: 0.9950 - val_accuracy: 0.5521
Epoch 4/100
380/380 [==============================] - 0s 226us/step - loss: 0.8759 - accuracy: 0.6395 - val_loss: 0.9492 - val_accuracy: 0.5276
Epoch 5/100
380/380 [==============================] - 0s 177us/step - loss: 0.7825 - accuracy: 0.6737 - val_loss: 0.9549 - val_accuracy: 0.5644
Epoch 6/100
380/380 [==============================] - 0s 167us/step - loss: 0.7440 - accuracy: 0.6789 - val_loss: 0.8881 - val_accuracy: 0.6135
Epoch 7/100
380/380 [==============================] - 0s 181us/step - loss: 0.7032 

Epoch 57/100
380/380 [==============================] - 0s 186us/step - loss: 0.1509 - accuracy: 0.9579 - val_loss: 0.4994 - val_accuracy: 0.8466
Epoch 58/100
380/380 [==============================] - 0s 167us/step - loss: 0.1578 - accuracy: 0.9368 - val_loss: 0.6018 - val_accuracy: 0.8282
Epoch 59/100
380/380 [==============================] - 0s 156us/step - loss: 0.1480 - accuracy: 0.9289 - val_loss: 0.4710 - val_accuracy: 0.8650
Epoch 60/100
380/380 [==============================] - 0s 123us/step - loss: 0.1337 - accuracy: 0.9474 - val_loss: 0.4916 - val_accuracy: 0.8466
Epoch 61/100
380/380 [==============================] - 0s 116us/step - loss: 0.1238 - accuracy: 0.9526 - val_loss: 0.4140 - val_accuracy: 0.8834
Epoch 62/100
380/380 [==============================] - 0s 172us/step - loss: 0.1190 - accuracy: 0.9579 - val_loss: 0.4955 - val_accuracy: 0.8589
Epoch 63/100
380/380 [==============================] - 0s 188us/step - loss: 0.1504 - accuracy: 0.9395 - val_loss: 0.5107 -

In [121]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

163/163 [==============================] - 0s 155us/step
over-sampling test accuracy: 89.57%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2,
       0, 1, 1, 1, 2, 0, 0, 0, 0, 2, 2, 1, 2, 0, 0, 2, 0, 0, 0, 1, 1, 1,
       1, 1, 2, 0, 0, 0, 1, 1, 1, 2, 2, 1, 2, 1, 1, 0, 2, 1, 0, 2, 2, 1,
       2, 0, 2, 2, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 0, 2, 1, 2, 2, 0, 2,
       0, 0, 0, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 0, 2, 1, 2, 2, 0, 2, 1, 0,
       2, 0, 1, 1, 0, 1, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 0, 1, 0,
       1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 0, 1, 1, 0, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,SR2852,2,2
1,CFBREBSa138,0,0
2,BCH-SA-12,0,0
3,EUH13,0,0
4,EUH13,0,0
...,...,...,...
158,NRS036,1,1
159,CA105,1,1
160,CFBRSa51,1,1
161,NRS102,1,1


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,1.275610e-05,0.000159,0.999829
1,9.952727e-01,0.000311,0.004416
2,9.997271e-01,0.000167,0.000106
3,9.994013e-01,0.000506,0.000093
4,9.994013e-01,0.000506,0.000093
...,...,...,...
158,7.718902e-04,0.952838,0.046390
159,4.223498e-04,0.997404,0.002173
160,1.017573e-03,0.928870,0.070113
161,6.538353e-06,0.978036,0.021957


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p11pST.csv", index = False,
         header=None)

In [125]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 202us/step - loss: 0.0714 - accuracy: 0.9711 - val_loss: 0.4821 - val_accuracy: 0.8650
Epoch 2/100
380/380 [==============================] - 0s 175us/step - loss: 0.0804 - accuracy: 0.9711 - val_loss: 0.2909 - val_accuracy: 0.8896
Epoch 3/100
380/380 [==============================] - 0s 152us/step - loss: 0.0902 - accuracy: 0.9579 - val_loss: 0.3454 - val_accuracy: 0.8957
Epoch 4/100
380/380 [==============================] - 0s 145us/step - loss: 0.0978 - accuracy: 0.9632 - val_loss: 0.5128 - val_accuracy: 0.8650
Epoch 5/100
380/380 [==============================] - 0s 151us/step - loss: 0.1002 - accuracy: 0.9579 - val_loss: 0.4730 - val_accuracy: 0.8712
Epoch 6/100
380/380 [==============================] - 0s 155us/step - loss: 0.0811 - accuracy: 0.9684 - val_loss: 0.3803 - val_accuracy: 0.9080
Epoch 7/100
380/380 [==============================] - 0s 167us/step - loss: 0.0771 

Epoch 57/100
380/380 [==============================] - 0s 156us/step - loss: 0.0548 - accuracy: 0.9842 - val_loss: 0.3985 - val_accuracy: 0.8957
Epoch 58/100
380/380 [==============================] - 0s 140us/step - loss: 0.0470 - accuracy: 0.9842 - val_loss: 0.4347 - val_accuracy: 0.8834
Epoch 59/100
380/380 [==============================] - 0s 111us/step - loss: 0.0599 - accuracy: 0.9816 - val_loss: 0.5595 - val_accuracy: 0.8712
Epoch 60/100
380/380 [==============================] - 0s 117us/step - loss: 0.0883 - accuracy: 0.9500 - val_loss: 0.5781 - val_accuracy: 0.8221
Epoch 61/100
380/380 [==============================] - 0s 116us/step - loss: 0.0641 - accuracy: 0.9816 - val_loss: 0.3750 - val_accuracy: 0.9018
Epoch 62/100
380/380 [==============================] - 0s 107us/step - loss: 0.0468 - accuracy: 0.9842 - val_loss: 0.4964 - val_accuracy: 0.8834
Epoch 63/100
380/380 [==============================] - 0s 120us/step - loss: 0.0503 - accuracy: 0.9737 - val_loss: 0.3606 -

In [126]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 96.89%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='p11presabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[1.27560950e-05, 1.58516250e-04, 9.99828700e-01],
       [9.95272700e-01, 3.11062000e-04, 4.41623600e-03],
       [9.99727100e-01, 1.66907030e-04, 1.06122614e-04],
       [9.99401330e-01, 5.05912930e-04, 9.28402950e-05],
       [9.99401330e-01, 5.05912930e-04, 9.28402950e-05],
       [1.28870890e-03, 9.54523270e-01, 4.41879480e-02],
       [9.25535000e-01, 4.42059040e-02, 3.02591120e-02],
       [8.23833430e-07, 2.01525520e-03, 9.97983930e-01],
       [9.72353200e-01, 2.21995860e-02, 5.44721000e-03],
       [9.99486450e-01, 5.46639060e-05, 4.58875580e-04],
       [2.89341550e-03, 8.20992230e-01, 1.76114320e-01],
       [9.15407360e-01, 5.15081920e-02, 3.30844630e-02],
       [2.94101200e-05, 6.73784640e-03, 9.93232700e-01],
       [9.72353200e-01, 2.21995860e-02, 5.44721000e-03],
       [1.47318420e-03, 1.32832410e-01, 8.65694400e-01],
       [9.95265360e-01, 1.51526980e-04, 4.58311940e-03],
       [9.72353200e-01, 2.21995860e-02, 5.44721000e-03],
       [9.99595700e-01, 8.14329

In [53]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9672772577971355

In [54]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9672772577971355

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9700645599728168

In [56]:
np.std(ovos)

0.005234214597552001

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9700645599728168

In [58]:
np.std(ovrs)

0.005234214597552001

In [127]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [128]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 90.34%


In [129]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.00796955305432868


In [130]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [131]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 97.15%


In [132]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.004326739
